# preprocess dataset

In [1]:
# import json
# path_codex="/media/data/flowers/OpenELM/Codex_PAPER_1M_iter_0.txt"
# # with open(path_codex, 'r') as f:
# #     codex = f.read()
#     # codex = [x.strip() for x in codex]
# f = open(path_codex, 'r')
# out=f.read()
# f.close()

# path_save="/media/data/flowers/OpenELM/codex_dataset.json"
# def format_dataset(puzzles):
#     puzzles_prompt=""
#     for i in range(len(puzzles)):
#         puzzles_prompt += f"Puzzle {i}:\n"+puzzles[i]+"\n"

#     return puzzles_prompt
# n_fewshot=4
# dataset =[]
# split_out=out.split("\nassert f(g())\n\n")
# split_out = [x+"\nassert f(g())\n" for x in split_out if x]
# split_out = split_out[:-1]
# for idx in range(0,len(split_out),n_fewshot):
#     split_out_n_fs = split_out[idx:idx+n_fewshot]
#     if len(split_out_n_fs)==n_fewshot:
#         dataset.append(format_dataset(split_out_n_fs))
# dataset_json= [{'text': puzz} for puzz in dataset]
# with open(path_save, 'w') as outfile:
#     json.dump(dataset_json, outfile,indent=4)

In [1]:
# test model
import os

os.environ['HF_DATASETS_CACHE'] = "/projets/flowers/julien/hf/datasets"
os.environ['TRANSFORMERS_CACHE'] = "/projets/flowers/julien/models/"
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig,TrainingArguments

from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
# from trl import SFTTrainer
# from datasets import load_dataset


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /projets/flowers/julien/miniconda3/envs/llm/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /projets/flowers/julien/miniconda3/envs/llm/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/projets/flowers/julien/miniconda3/envs/llm/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /projets/flowers/julien/miniconda3/envs/llm did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/projets/flowers/julien/miniconda3/envs/llm/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('3128'), PosixPath('http'), PosixPath('//proxy.plafrim.cluster')}
  warn(msg)
/projets/flowers/julien/miniconda3/envs/llm/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/home/jpourcel/.guix-profile/lib/locale')}
  warn(msg)
/projets/flowers/julien/miniconda3/envs/llm/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: 

# base training

## train

In [ ]:

# train on puzzle generated (except P3 trainset)
#https://huggingface.co/docs/trl/main/en/sft_trainer


# testset which is in fact the trainset from P3 dataset
# path_test = "/media/data/flowers/OpenELM/preprocess_p3.json"

 
# /!\ need to remove that, just for testing purpose /!\

# path_train = path_test 
# ================================================

# test_set = load_dataset("json", data_files=path_test)
model_id = "Salesforce/codegen-350M-mono"#"bigcode/starcoderbase-1b"  "Salesforce/codegen-350M-mono" # mono is better
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="bfloat16"#torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(model_id,quantization_config=bnb_config, device_map="auto",trust_remote_code=True)
model.config.use_cache = False



peft_config = LoraConfig(
    r=32,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["qkv_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)



# train

In [ ]:
# def formatting_func(example):
#     # text = f"### Question: {example['question']}\n ### Answer: {example['answer']}"
#     text = example["text"]
#     return text
path_train= "/media/data/flowers/OpenELM/logs/elm/23-08-22_12:36/step_9/save_all.json" # put path here
path_save="/media/data/flowers/OpenELM/codex_dataset.json"
dataset = load_dataset("json", data_files=path_save,split="train")
dataset = dataset.train_test_split(test_size=0.001)

training_arguments=TrainingArguments(
    per_device_train_batch_size=6,
    per_device_eval_batch_size=6,
    evaluation_strategy="steps",
    gradient_accumulation_steps=2,
    do_eval=True,
    # warmup_steps=2,
    warmup_ratio=0.01,
    max_steps = 2000,
    eval_steps=10,
    weight_decay=0.001,
    learning_rate=2e-4,
    
    # fp16=True,
    logging_steps=1,
    output_dir="outputs",
    optim="adamw_hf",#"paged_adamw_32bit",
    max_grad_norm=0.3,
    group_by_length=True
)

trainer = SFTTrainer(
    model,#"EleutherAI/gpt-neo-125m",
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    dataset_text_field="text",
    peft_config=peft_config,
    args= training_arguments
)

trainer.train()
model.config.use_cache = True

In [ ]:
output_dir = "/media/data/flowers/OpenELM/models/codegen-QDAIF-smart"
trainer.save_model(output_dir)
from peft import AutoPeftModelForCausalLM
del model
model = AutoPeftModelForCausalLM.from_pretrained(output_dir, device_map="auto", torch_dtype=torch.bfloat16)
model = model.merge_and_unload()
model.eval()
model.config.use_cache = True

# test

In [3]:
import numpy as np
def test_puzzle(test_fg):
    test_fg= "from typing import *\n"+test_fg
    try:
        exec(test_fg)
        return True
    except:
        return False
    
prompt_solve_puzzle='''You will be given a function and its docstring. Respond only in code with a correct, efficient implementation of the function.
You need to generate the correct solutions (g), for the Problem 3 that satisfies the condition f(g()) == True.
Problem 0:
```
def f(stamps: List[int], target=80, max_stamps=4, options=[10, 32, 8]) -> bool:
    """Find a selection of at most max_stamps stamps whose total worth is the target value."""
    for s in stamps:
        assert s in options
    return len(stamps) <= max_stamps and sum(stamps) == target
```
Solution 0:
```
def g(target = 80, max_stamps = 4, options = [10, 32, 8]):
    from itertools import combinations_with_replacement
    for n in range(max_stamps + 1):
        for c in combinations_with_replacement(options, n):
            if sum(c) == target:
                return list(c)
assert f(g())
```
Problem 1:
```
from typing import*
def f(ans: List[List[int]], target=2) -> bool:
    """
    Find a list of pairs of integers where the number of pairs in which the second number is more than
    two greater than the first number is a given constant
    """
    for i in range(len(ans)):
        a, b = ans[i]
        if b - a >= 2:
            target -= 1
    return target == 0
```
Solution 1:
```
def g(target = 2):
    return [[0, 2]] * target 
assert f(g()) == True
```
Problem 2:
```
def f(n: int, v=313946483, w=806690290) -> bool:
    """Find the smallest n such that if v is tripled n times and w is doubled n times, v exceeds w."""
    for i in range(n):
        assert v <= w
        v *= 3
        w *= 2
    return v > w
```
Solution 2:
```
def g(v = 313946483, w = 806690290):
    i = 0
    while v <= w:
        v *= 3
        w *= 2
        i += 1
    return i 
assert f(g()) == True
```
Problem 3:
```
{pb}
```
Solution 3:
```
'''


def pass_at_k(n, c, k):
    """
    Adapted from "Evaluating Large Language Models Trained on Code" (https://arxiv.org/abs/2107.03374)

    :param n: total number of samples
    :param c: number of correct samples
    :param k: k in pass@k
    """
    assert n >= k
    if n - c < k:
        return 1.0
    return 1.0 - np.prod(1.0 - k / np.arange(n - c + 1, n + 1))

In [4]:
model_id="codellama/CodeLlama-7b-Python-hf"
# del model
# quantization_config = BitsAndBytesConfig(
#    load_in_4bit=True,
#    bnb_4bit_compute_dtype=torch.bfloat16
# )

tokenizer = AutoTokenizer.from_pretrained(model_id,trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,

    # quantization_config=quantization_config,
    device_map="auto",
    trust_remote_code=True
)
tokenizer.padding_side='left'
tokenizer.pad_token = tokenizer.eos_token
model.eval()
# model.config.use_cache = True
model=torch.compile(model)
# prompt = 'def remove_non_ascii(s: str) -> str:\n""" '
# inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

# output = model.generate(
#     inputs["input_ids"],
#     max_new_tokens=64,
#     do_sample=True,
#     top_p=0.9,
#     temperature=0.1,
# )

# output = output[0].to("cpu")
# print(tokenizer.decode(output))




Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
import json
from tqdm import tqdm
path_trainset="/projets/flowers/julien/OpenELM/src/openelm/utils/preprocess_p3_emb.json"
with open(path_trainset, 'r') as f:
    trainset = json.load(f)
    
list_trainset= [x["program_str"] for x in trainset]


In [24]:
torch._dynamo.config.suppress_errors = True
list_puzzle_correct=[]
correct_puzz=0
curr_idx=0

In [25]:
num_return_sequences=10 #n_try
list_passk=[]
list_puzzle=[]
with torch.no_grad():
    
    for idx in tqdm(range(curr_idx,len(list_trainset))): #len(dataset["test"])
        curr_idx=idx
        # idx=0
        print(f"\n\n============ idx {idx} ==================\n")
        flag=True
        attempt=0
        list_puzzle_idx=[]
        while flag and attempt<50:
            attempt+=1
            try:
                puzzle= list_trainset[idx]
                prompt_f = puzzle.split("def g(")[0]
                prompt = prompt_solve_puzzle.format(pb=prompt_f)
                inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
                outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), attention_mask=inputs["attention_mask"], max_new_tokens=256,do_sample=True, temperature=0.8,num_return_sequences=num_return_sequences)
                len_prompt=inputs["input_ids"][0].shape[0]
                flag=False
                cor_puz=0
                
                for i in range(len(outputs)):
                    
                    # out ="def g("
                    out = tokenizer.decode(outputs[i][len_prompt:], skip_special_tokens=True)
                    # print("output gene[rated:",out)
                    
                    extract_g=out.split("```")[0].split("assert")[0]
                    extract_g = extract_g+"\nassert f(g()) == True\n"
                    test_fg= prompt_f+extract_g 
                    list_puzzle.append(test_fg)
                    if i<1:
                        print("\n-------------------\n")
                        print(test_fg)
                    if test_puzzle(test_fg):
                        if cor_puz==0:
                            print("\n-------------------\n")
                            print(test_fg)
                        cor_puz+=1

                        list_puzzle_idx.append(test_fg)
                    n_sample, n_correct=10,cor_puz
                    pass_k = pass_at_k(n_sample, n_correct, k=10)
            except Exception as e:
                print("pb when generating puzzles")
                print(str(e))
                pass_k=0.
                pass


        list_puzzle_correct.append(list_puzzle_idx)   

        if cor_puz>=1:
            print("\n=================\n")
            print("correct puzzle",correct_puzz)
            correct_puzz+=1
        list_passk.append(pass_k)

        print(f"correct puzzles: {correct_puzz}/{idx+1}")

  0%|          | 0/536 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




============ idx 0 ==================



  0%|          | 1/536 [00:11<1:43:15, 11.58s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



-------------------

def f(s: str) -> bool:
    """Find a string with 1000 'o's but no two adjacent 'o's."""
    return s.count('o') == 1000 and s.count('oo') == 0
def g(s = "o"*1000):
    return s

assert f(g()) == True

correct puzzles: 0/1


============ idx 1 ==================



  0%|          | 2/536 [00:23<1:42:46, 11.55s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...


-------------------

def f(li: List[int]) -> bool:
    """Find a list integers such that the integer i occurs i times, for i = 0, 1, 2, ..., 9."""
    return all([li.count(i) == i for i in range(10)])
def g():
    return [0] * 10

assert f(g()) == True

correct puzzles: 0/3


============ idx 3 ==================


-------------------

def f(s: str) -> bool:
    """Find a three-digit pattern  that occurs more than 8 times in the decimal representation of 8^2888."""
    return str(8 ** 2888).count(s) > 8 and len(s) == 3
def g():
    s = str(8 ** 2888)    
    return s[0], s[1], s[2]

assert f(g()) == True



  1%|          | 4/536 [04:01<13:07:53, 88.86s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


correct puzzles: 0/4


============ idx 4 ==================



  1%|          | 5/536 [04:13<8:59:39, 60.98s/it] Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



-------------------

def f(li: List[int]) -> bool:
    """
    Find a way to rearrange the letters in the pangram "The quick brown fox jumps over the lazy dog" to get
    the pangram "The five boxing wizards jump quickly". The answer should be represented as a list of index
    mappings.
    """
    return ['The quick brown fox jumps over the lazy dog'[i] for i in li] == list('The five boxing wizards jump quickly')
def g():
    li = [17, 27, 12, 10, 5, 13, 1, 3, 15, 8, 16, 21, 20, 11, 6, 18, 19, 25, 4, 2, 22, 14, 23, 9, 14, 7, 18, 24, 13]
    return li 

assert f(g()) == True

correct puzzles: 0/5


============ idx 5 ==================



In [7]:
print(list_trainset[0])

def f(s: str) -> bool:
    """Find a string with 1000 'o's but no two adjacent 'o's."""
    return s.count('o') == 1000 and s.count('oo') == 0
def g():
    return ('h' + 'o') * 1000
assert f(g())


In [13]:
torch._dynamo.config.suppress_errors = True
with torch.no_grad():
    idx=0
    puzzle= list_trainset[idx:idx+4]
    input_tok=tokenizer(puzzle, return_tensors='pt', padding=True).to("cuda")
    out_model = model.generate(**input_tok, max_new_tokens=256,do_sample=True, temperature=0.5)
    outokenizer.batch_decode(outputs, skip_special_tokens=True)



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [22]:
len(input_tok["input_ids"])
input_tok["input_ids"][0]
out = tokenizer.decode(out_model[0], skip_special_tokens=True)

In [23]:
out

'def f(s: str) -> bool:\n    """Find a string with 1000 \'o\'s but no two adjacent \'o\'s."""\n    return s.count(\'o\') == 1000 and s.count(\'oo\') == 0\ndef g():\n    return (\'h\' + \'o\') * 1000\nassert f(g())\n'

In [ ]:
# PASS 4

b=4
num_return_sequences=10 #n_try
list_passk=[]
list_puzzle=[]
with torch.no_grad():
    
    for idx in tqdm(range(0,curr_idx,len(list_trainset)),bs): #len(dataset["test"])
        curr_idx=idx
        # idx=0
        print(f"\n\n============ idx {idx} ==================\n")
        flag=True
        attempt=0
        list_puzzle_idx=[]
        while flag and attempt<10:
            attempt+=1
            try:
                puzzle= list_trainset[idx]
                prompt_f = puzzle.split("def g(")[0]
                prompt = prompt_solve_puzzle.format(pb=prompt_f)
                inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
                outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), attention_mask=inputs["attention_mask"], max_new_tokens=256,do_sample=True, temperature=0.5)
                len_prompt=inputs["input_ids"][0].shape[0]
                flag=False
                cor_puz=0
                
                for i in range(len(outputs)):
                    
                    # out ="def g("
                    out = tokenizer.decode(outputs[i][len_prompt:], skip_special_tokens=True)
                    # print("output gene[rated:",out)
                    
                    extract_g=out.split("```")[0].split("assert")[0]
                    extract_g = extract_g+"\nassert f(g()) == True\n"
                    test_fg= prompt_f+extract_g 
                    list_puzzle.append(test_fg)
                    if i<1:
                        print("\n-------------------\n")
                        print(test_fg)
                    if test_puzzle(test_fg):
                        if cor_puz==0:
                            print("\n-------------------\n")
                            print(test_fg)
                        cor_puz+=1

                        list_puzzle_idx.append(test_fg)
                    n_sample, n_correct=10,cor_puz
                    pass_k = pass_at_k(n_sample, n_correct, k=10)
            except Exception as e:
                print("pb when generating puzzles")
                print(str(e))
                pass_k=0.
                pass


        list_puzzle_correct.append(list_puzzle_idx)   

        if cor_puz>=1:
            print("\n=================\n")
            print("correct puzzle",correct_puzz)
            correct_puzz+=1
        list_passk.append(pass_k)

        print(f"correct puzzles: {correct_puzz}/{idx+1}")

# train only on completion

In [ ]:
import os
os.environ['TRANSFORMERS_CACHE'] = "/media/data/flowers/OpenELM/models"
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig,TrainingArguments
from peft import LoraConfig
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from datasets import load_dataset

# train on puzzle generated (except P3 trainset)
#https://huggingface.co/docs/trl/main/en/sft_trainer


path_train= "/media/data/flowers/OpenELM/logs/elm/23-08-22_12:36/step_9/save_all.json" # put path here
dataset = load_dataset("json", data_files=path_train, split="train")


model_id = "Salesforce/codegen2-1B"
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="bfloat16"#torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(model_id,quantization_config=bnb_config, device_map="auto",trust_remote_code=True)
model.config.use_cache = False
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

import json



path = "/media/data/flowers/OpenELM/preprocess_p3.json" 

# path = "/media/data/flowers/OpenELM/logs/elm/23-08-22_12:36/step_9/save_all.json" # QDAIF smart
with open(path, 'r') as f:
    data = json.load(f)
f=data[0]["program_str"].split("def g")[0]
prompt=""
n_fewshot=3
for i in range(1,1+n_fewshot):
    prompt+=f"Puzzle {i}:\n"+data[i]["program_str"]+"\n\n"
# text = prompt+ f"Puzzle {n_fewshot}:\n"+ f +"\ndef g():"

def formatting_prompts_func(example,prompt=prompt):
    output_texts = []
    text = f"{prompt}\nPuzzle 4:\n {example['program_str'][i]}"
    output_texts.append(text)
    return output_texts

response_template = "Puzzle 4:\n"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)


tokenizer.padding_side = "right"


training_arguments=TrainingArguments(
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    # evaluation_strategy="epoch", #"steps",
    gradient_accumulation_steps=4,
    # warmup_steps=2,
    warmup_ratio=0.2,
    max_steps = 12,
    weight_decay=0.001,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=1,
    output_dir="outputs",
    optim="adamw_hf",#"paged_adamw_32bit",
    max_grad_norm=0.3,
    # group_by_length=True
)

trainer = SFTTrainer(
    model,#"EleutherAI/gpt-neo-125m",
    train_dataset=dataset,
    formatting_func=formatting_prompts_func,
    data_collator=collator,
    peft_config=peft_config,
    args=training_arguments
)

trainer.train()

output_dir = "/media/data/flowers/OpenELM/models/codegen-QDAIF-smart"
trainer.model.save_pretrained(output_dir)

In [ ]:
import numpy as np
out=[15,17,16.5,16.,16.,17,16.25,17.3,15]
len(out)
out.sort(reverse=True)
np.mean(out[:8])

In [ ]:
import json
path = "/media/data/flowers/OpenELM/preprocess_p3.json" 

# path = "/media/data/flowers/OpenELM/logs/elm/23-08-22_12:36/step_9/save_all.json" # QDAIF smart
with open(path, 'r') as f:
    data = json.load(f)

f=data[0]["program_str"].split("def g")[0]
prompt=""
n_fewshot=3
for i in range(1,1+n_fewshot):
    prompt+=f"Puzzle {i}:\n"+data[i]["program_str"]+"\n\n"
    # prompt+=data[i]["program_str"]+"\n"
tag_response = f"Puzzle {n_fewshot+1}:\n"
tag_eo_completion = f"Puzzle"
text = prompt+ f"Puzzle {n_fewshot+1}:\n"+ f #+"def g():"
# text = f#prompt+ f 


with torch.no_grad():
    inputs = tokenizer(text, return_tensors="pt").to("cuda")
    outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), attention_mask=inputs["attention_mask"], max_new_tokens=256,do_sample=True,temperature=1.)
    gen_text= tokenizer.decode(outputs[0], skip_special_tokens=True)
    print()

In [ ]:
model

In [ ]:
gen_text.split(tag_response)[1].split(tag_eo_completion)[0].strip()

In [ ]:
texts = [text]*5
encoding = tokenizer(texts, padding=True, return_tensors='pt').to("cuda")
with torch.no_grad():
    generated_ids = model.generate(**encoding,max_new_tokens=256,do_sample=True,temperature=1.)
generated_texts = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)


In [ ]:
filter_text = [puzz.split(tag_response)[1].split(tag_eo_completion)[0].strip() for puzz in generated_texts]

In [ ]:
[print(puzz+"\n") for puzz in filter_text]

# other stuff

In [ ]:
import os
os.environ['TRANSFORMERS_CACHE'] = "/media/data/flowers/OpenELM/models"
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
# from huggingface_hub import notebook_login
# notebook_login()

peft=True
model_id = "Salesforce/codegen-350M-mono"#"bigcode/starcoderbase-1b"#"Salesforce/codegen-350M-mono" # mono is better
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16"#torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
if peft:
    model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto",trust_remote_code=True)
else:
    model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto",trust_remote_code=True)
    
if peft:
    # model.gradient_checkpointing_enable()
    model = prepare_model_for_kbit_training(model)

    def print_trainable_parameters(model):
        """
        Prints the number of trainable parameters in the model.
        """
        trainable_params = 0
        all_param = 0
        for _, param in model.named_parameters():
            all_param += param.numel()
            if param.requires_grad:
                trainable_params += param.numel()
        print(
            f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
        )
    print(model)


    config = LoraConfig(
        r=16, 
        lora_alpha=32, 
        target_modules=["qkv_proj"],# "lm_head" s["c_attn","c_proj"],# codegen ["qkv_proj"],#["query_key_value"],  
        lora_dropout=0.01, 
        bias="none", 
        task_type="CAUSAL_LM",
        # layers_to_transform=[i for i in range(5,15)]
    )

    model = get_peft_model(model, config)
    print_trainable_parameters(model)

from datasets import load_dataset
# train on puzzle generated (except P3 trainset)

path_train= "/media/data/flowers/OpenELM/logs/elm/23-08-22_12:36/step_9/save_all.json" # put path here
squad_it_dataset = load_dataset("json", data_files=path_train)

# testset which is in fact the trainset from P3 dataset
path_test = "/media/data/flowers/OpenELM/preprocess_p3.json"

 
# /!\ need to remove that, just for testing purpose /!\

path_train = path_test 
# ================================================

test_set = load_dataset("json", data_files=path_test)

def filter_correct_puzz(example):
    return (
        example["fitness"] >0.
    )
# only keep correct puzzles
correct_dataset = squad_it_dataset.filter(filter_correct_puzz)

tokenized_datasets = correct_dataset.map(lambda samples: tokenizer(samples["program_str"]), batched=True)
tok_test_set = test_set.map(lambda samples: tokenizer(samples["program_str"]), batched=True)

In [ ]:
import transformers

# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_datasets["train"],
    eval_dataset = tok_test_set,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        evaluation_strategy="epoch", #"steps",
        gradient_accumulation_steps=2,
        # warmup_steps=2,
        warmup_ratio=0.1,
        num_train_epochs = 2,
        # max_steps=10,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_32bit",
        max_grad_norm=0.3,
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

In [ ]:
path_model = "/media/data/flowers/OpenELM/models/codegen-QDAIF-smart"
trainer.model.save_pretrained(path_model)


In [ ]:
from peft import LoraConfig, PeftModel

base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
model = PeftModel.from_pretrained(base_model, path_model)
model = model.merge_and_unload()



In [ ]:
from transformers import pipeline
import json



path = "/media/data/flowers/OpenELM/preprocess_p3.json" 

# path = "/media/data/flowers/OpenELM/logs/elm/23-08-22_12:36/step_9/save_all.json" # QDAIF smart
with open(path, 'r') as f:
    data = json.load(f)
f=data[0]["program_str"].split("def g")[0]
prompt=""
n_fewshot=2
for i in range(1,n_fewshot):
    prompt+=f"Puzzle {i}:\n"+data[i]["program_str"]+"\n\n"
text = prompt+ f"Puzzle {n_fewshot}:\n"+ f +"\ndef g():"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=512)
result = pipe(text)
print(result[0]['generated_text'])

In [ ]:
import json
path = "/media/data/flowers/OpenELM/preprocess_p3.json" 

# path = "/media/data/flowers/OpenELM/logs/elm/23-08-22_12:36/step_9/save_all.json" # QDAIF smart
with open(path, 'r') as f:
    data = json.load(f)

f=data[11]["program_str"].split("def g")[0]
prompt=""
n_fewshot=1
for i in range(n_fewshot):
    prompt+=f"Puzzle {i}:\n"+data[i]["program_str"]+"\n\n"
text = prompt+ f"Puzzle {n_fewshot}:\n"+ f #+"def g():"
# text = f


with torch.no_grad():
    inputs = tokenizer(text, return_tensors="pt").to("cuda")
    outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), attention_mask=inputs["attention_mask"], max_new_tokens=256)

    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
model_id = "Salesforce/codegen2-1B"#"bigcode/starcoderbase-1b"#"Salesforce/codegen-350M-mono" # mono is better
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto",trust_remote_code=True)
f=data[11]["program_str"].split("def g")[0]
prompt=""
n_fewshot=2
for i in range(n_fewshot):
    prompt+=f"Puzzle {i}:\n"+data[i]["program_str"]+"\n\n"
text = prompt+ f"Puzzle {n_fewshot}:\n"+ f #+"def g():"
with torch.no_grad():
    inputs = tokenizer(text, return_tensors="pt").to("cuda")
    outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), attention_mask=inputs["attention_mask"], max_new_tokens=256)

    print(tokenizer.decode(outputs[0], skip_special_tokens=True))